In [2]:
import numpy as np
import math
# Module to display T/F matrix
import matplotlib.pyplot as plt
from matplotlib import ticker, cm
# Module for Unit testing
import pytest
# Module for convolutionnal decoder
import sk_dsp_comm.fec_conv as fec

In [5]:
my_data = np.genfromtxt('./tfMatrix.csv', delimiter=';')
mat_complex = my_data[:,0::2] +1j*my_data[:,1::2]

print(mat_complex.shape)

[[ 0.        +0.00000000e+00j  1.41246594+1.42228929e-02j
   1.41139197+5.68771522e-02j ...  1.41139197+5.68771522e-02j
   1.41246594+1.42228929e-02j  1.41253754+2.65874304e-13j]
 [ 0.        +0.00000000e+00j  1.41246594-1.42228929e-02j
   1.41139197-5.68771522e-02j ...  1.41139197-5.68771522e-02j
   1.41246594-1.42228929e-02j  1.41253754-2.65874304e-13j]
 [ 0.        +0.00000000e+00j -1.        +0.00000000e+00j
  -1.        +0.00000000e+00j ... -1.        +0.00000000e+00j
  -1.        +0.00000000e+00j -1.        +0.00000000e+00j]
 ...
 [ 0.        +0.00000000e+00j  0.9486833 -9.48683298e-01j
   0.31622777+3.16227766e-01j ... -0.9486833 -9.48683298e-01j
  -0.31622777-9.48683298e-01j  0.31622777+9.48683298e-01j]
 [ 0.        +0.00000000e+00j  0.31622777+3.16227766e-01j
   0.31622777+9.48683298e-01j ... -0.9486833 +9.48683298e-01j
  -0.9486833 +3.16227766e-01j  0.9486833 -9.48683298e-01j]
 [ 0.        +0.00000000e+00j  0.9486833 -3.16227766e-01j
   0.31622777-3.16227766e-01j ... -0.31622